In this notebook we will explore the daos census dataset to select which DAOs we want to run the recommender system against

In [ ]:
from pathlib import Path
import datetime as dt
import requests # To see if they are flagged (spam)
import requests_cache

import numpy as np
import pandas as pd

from tqdm.autonotebook import tqdm

import duckdb

tqdm.pandas()
requests_cache.install_cache(
    cache_control=True,
    use_cache_dir=True,
    expire_after=dt.timedelta(weeks=1),
    allowable_methods=['GET', 'POST'],
)

%load_ext sql
%config SqlMagic.autopandas = True

In [ ]:
KAGGLE_DATASET: str = 'daviddavo/daos-census-tfm'
AUX_PATH: Path = Path('~/Downloads/daos-census-tfm').expanduser()
CUTOFF_DATE = None

PLATFORMS_WITH_TEXT = ['aragon', 'daohaus', 'daostack', 'snapshot']
# Also calculate the number of proposals made in the last X weeks or whatever
# we will just run the recommender in a certain period, not in all the timespan
# so it doesn't make sense to use DAOs with lots of proposals but few of them
# where we will run the recommender, and other DAOs with not many proposals
# but a lot when we would run the recommender
CONSIDER_LAST_PERIOD_STR = '20w'
# This dictionary merges different deployments into one organization
ORGS_DICT: dict[str, list[str]] = {
    'dxDAO - xDXdao': ['dxDAO', 'xDXdao'],
    'Aave - Aavegotchi': ['Aave', 'Aavegotchi', 'AAVE'],
    'MetaCartel - MetaCartel Ventures': ['MetaCartel Ventures', 'MetaCartel xDai', 'MetaCartel DAO'],
}
CUSTOM_EXCLUSIONS = {
    'snapshot': [
        # Not used as a DAO, just bogus proposals
        'ppybbbbbb.eth',
        
        # Not transparent
        'vote.nftgoingtozero.eth',
    ]
}

In [ ]:
CONSIDER_LAST_PERIOD = pd.Timedelta(CONSIDER_LAST_PERIOD_STR)
CONSIDER_LAST_PERIOD

## Download dataset if it does not exist

In [ ]:
import kaggle

In [ ]:
if not AUX_PATH.exists():
    kaggle.api.dataset_download_cli(KAGGLE_DATASET, path=AUX_PATH, unzip=True)

## Processing the dataset

In [ ]:
def _list2sql(lst: list[str]) -> str:
    return "".join(["(", ", ".join(map("'{}'".format, lst)), ")"])

def _gen_orgs_query(parquet: Path) -> str:
    _casestr = "    WHEN name IN {caselst} THEN '{orgname}'"

    _cases = "\n".join(_casestr.format(
        orgname=orgname,
        caselst=_list2sql(caselst),
    ) for orgname, caselst in ORGS_DICT.items())
    
    return f"""
CREATE VIEW deployments AS
SELECT * EXCLUDE (name),
    name AS deployment_name,
    CASE 
{_cases}
    ELSE name
    END AS name
FROM parquet_scan('{parquet}')
    """

### Import from the parquets

In [ ]:
db = duckdb.connect(database=':memory:', read_only=False)
# db.execute(_gen_orgs_query(AUX_PATH / 'deployments.parquet'))
_cond_date = ""
if CUTOFF_DATE:
    _cond_date = f"WHERE date <= '{CUTOFF_DATE.isoformat()}'"
print(_cond_date)

db.execute("CREATE VIEW deployments AS SELECT * FROM parquet_scan('{}')".format(AUX_PATH / "deployments.parquet"))
db.execute("CREATE VIEW votes AS SELECT * FROM parquet_scan('{}') {}".format(AUX_PATH / "votes.parquet", _cond_date))
db.execute("CREATE VIEW proposals AS SELECT * FROM parquet_scan('{}') {}".format(AUX_PATH / "proposals-text.parquet", _cond_date))

%sql db --alias duckdb

In [ ]:
db.execute("SHOW TABLES").fetchdf()

In [ ]:
%%sql
SHOW TABLES

In [ ]:
%%sql
SELECT * FROM duckdb_columns() WHERE table_name IN ('votes')

In [ ]:
%%sql
SELECT *
FROM deployments
WHERE name = 'Aura Finance'

In [ ]:
%%sql
SELECT COUNT(DISTINCT votes.proposal_id)
FROM votes
    JOIN deployments ON deployment_id = deployments.id
WHERE name = 'Aura Finance'

In [ ]:
_casestr = "WHEN name IN {caselst} THEN '{orgname}'"

_cases = "\n".join(_casestr.format(
    orgname=orgname,
    caselst=_list2sql(caselst),
) for orgname, caselst in ORGS_DICT.items())
print(_cases)

# Get "orgs"

We merge multiple deployments into one "organization", if they have the same name or if they are on the `ORGS_DICT`. We also obtain some useful data.

In [ ]:
%%sql
CREATE OR REPLACE VIEW orgs AS
    WITH G AS (WITH Gv AS (
            SELECT *,
                CASE
                    {{_cases}}
                    ELSE name
                END AS orgname
            FROM deployments
            LEFT JOIN votes ON (deployments.id = votes.deployment_id)
            WHERE 
                platform IN {{_list2sql(PLATFORMS_WITH_TEXT)}} AND
                proposals_count > 10
        )
        SELECT 
            orgname AS name,
            COUNT(DISTINCT Gv.id) AS n_deploys,
            LIST(DISTINCT Gv.id) AS deploys,
            LIST(DISTINCT Gv.platform) AS platforms,
            LIST(DISTINCT Gv.platform_deployment_id) AS platform_ids,
            -- Proposals with at least a vote!
            COUNT(DISTINCT Gv.proposal_id) AS proposals_count,
            -- Users with at least a vote!
            COUNT(DISTINCT Gv.voter) AS voters_count,
            MAX(Gv.date) AS last_vote,
            COUNT(*) AS votes_count,
        FROM Gv
        GROUP BY Gv.orgname
    )
    SELECT
        *,
        proposals_count + voters_count AS nodes,
        2*(votes_count)/((voters_count + proposals_count)*(voters_count + proposals_count-1)) AS density_naive,
        2*(votes_count)/(voters_count*proposals_count) AS density,
        votes_count/proposals_count AS vpp, 
        votes_count/voters_count AS vpv,
    FROM G
    ORDER BY proposals_count DESC

In [ ]:
%%sql
SELECT *
FROM orgs
WHERE name = 'Aura Finance'

In [ ]:
%%sql df_all <<
SELECT *
FROM orgs
WHERE name is NOT NULL
    AND proposals_count > 100

## Check which daos were marked as SPAM

In [ ]:
def _get_space_flagged(space: str) -> bool:
    if space in CUSTOM_EXCLUSIONS['snapshot']:
        return 1
    
    r = requests.post("https://hub.snapshot.org/graphql", json={
        'query': """
        query ($id: String!) {
            space(id: $id) { flagged }
        }
        """,
        'variables': {'id': space},
    })
    try:
        return int(r.json()['data']['space']['flagged'])
    except:
        print(f"Error with {space}")
        raise

print(_get_space_flagged('gm365.eth'))
print(_get_space_flagged('ppybbbbbb.eth'))

In [ ]:
def _get_org_flagged(row):
    if 'snapshot' not in row['platforms']:
        return 0

    return sum( _get_space_flagged(d) for d in row['platform_ids'] if d.endswith('.eth') )

df_all['flagged'] = df_all.progress_apply(_get_org_flagged, axis=1)

In [ ]:
df_all_final = df_all[(df_all['flagged'] == 0) & (df_all['voters_count'] >= 20)].head(25).reset_index(drop=True)
df_all_final

## Put everything in a nice table

In [ ]:
cols = {
    'name': 'Name',
    'proposals_count': '# Prop.',
    'voters_count': '# Voters',
    'votes_count': '# Votes',
    'milli density': '‰ Density',
    'vpp': 'vpp',
    'vpv': 'vpv',
}

df_all_final['milli density'] = df_all_final['density']*1000
# msk = (df_all['flagged'] == 0) & (df_all['voters_count'] >= 20)
bof = df_all_final[cols.keys()]#.head(21)

style = (bof
    .style
    .format('{:.2f}', subset=bof.select_dtypes('float').columns)
    .format(thousands=' ', subset=bof.select_dtypes('int').columns)
    .relabel_index(list(cols.values()), axis='columns')
    .hide(axis='index')
)
display(style)
print(style.to_latex())

# Replicate the "orgs" thing only with "recent" proposals

In [ ]:
CONSIDER_LAST_PERIOD

In [ ]:
%%sql
CREATE OR REPLACE VIEW orgs_recent AS
    WITH G AS (WITH Gv AS (
            SELECT *,
                CASE
                    {{_cases}}
                    ELSE name
                END AS orgname
            FROM deployments
            LEFT JOIN (
                SELECT *
                FROM votes
                -- Check that the votes are recent
                WHERE date > ( (SELECT MAX(date) FROM votes) - (INTERVAL '{{CONSIDER_LAST_PERIOD.total_seconds()}}' SECOND))
            ) AS votes_recent ON (deployments.id = votes_recent.deployment_id)
            WHERE 
                platform IN {{_list2sql(PLATFORMS_WITH_TEXT)}} AND
                proposals_count > 10
        )
        SELECT 
            orgname AS name,
            COUNT(DISTINCT Gv.id) AS n_deploys,
            LIST(DISTINCT Gv.id) AS deploys,
            LIST(DISTINCT Gv.platform) AS platforms,
            LIST(DISTINCT Gv.platform_deployment_id) AS platform_ids,
            COUNT(DISTINCT Gv.proposal_id) AS proposals_count,
            COUNT(DISTINCT Gv.voter) AS voters_count,
            COUNT(*) AS votes_count,
        FROM Gv
        GROUP BY Gv.orgname
    )
    SELECT
        *,
        proposals_count + voters_count AS nodes,
        2*(votes_count)/((voters_count + proposals_count)*(voters_count + proposals_count-1)) AS density_naive,
        2*(votes_count)/(voters_count*proposals_count) AS density,
        votes_count/proposals_count AS vpp, 
        votes_count/voters_count AS vpv
    FROM G
    WHERE proposals_count > 0
    ORDER BY proposals_count DESC

In [ ]:
%%sql df_recent <<
SELECT *
FROM orgs_recent
WHERE proposals_count > 100
    AND name IS NOT NULL

In [ ]:
df_recent['flagged'] = df_recent.progress_apply(_get_org_flagged, axis=1)

In [ ]:
df_recent[(df_recent['flagged'] == 0) & (df_recent['voters_count'] >= 20)].head(20).reset_index(drop=True)

In [ ]:
df_recent_final = df_recent[(df_recent['flagged'] == 0) & (df_recent['voters_count'] >= 20) & ~(df_recent['name'].isin(df_all_final['name']))].head(20).reset_index(drop=True)
df_recent_final

### Put everything on a nice table

In [ ]:
cols = {
    'name': 'Name',
    'proposals_count': '# Prop.',
    'voters_count': '# Voters',
    'votes_count': '# Votes',
    'milli density': '‰ Density',
    'vpp': 'vpp',
    'vpv': 'vpv',
}

df_recent['milli density'] = df_recent['density']*1000
msk = (df_recent['flagged'] == 0) & (df_recent['voters_count'] >= 20)
bof = df_recent[msk][cols.keys()].head(20)

style = (bof
    .style
    .format('{:.2f}', subset=bof.select_dtypes('float').columns)
    .format(thousands=' ', subset=bof.select_dtypes('int').columns)
    .relabel_index(list(cols.values()), axis='columns')
    .hide(axis='index')
)
display(style)
print(style.to_latex())